In [1]:
import spacy
nlp_en = spacy.load('en')
nlp_de = spacy.load('de')
from spacy.en import English
parser = English()
import xml.etree.ElementTree as ET

## Load Reference Grammar

In [2]:
def create_prompt_response_map(gram):
    prompt_response_map = {}
    for prompt_unit in gram:
        key = None
        responses = []
        for prompt in prompt_unit:
            if prompt.tag == None:
                print("None")
            else:
                if(prompt.tag == 'prompt'):
                    key = prompt.text
                elif(prompt.tag == 'response'):
                    responses.append(prompt.text)
        prompt_response_map[key] = responses
    return prompt_response_map

def read_grammar_and_create_map(file):
    tree = ET.parse(file)
    grammar = tree.getroot()
    return create_prompt_response_map(grammar)

def get_sag_prompts(prompt_response_map):
    sag_prompts = {}
    for key in prompt_response_map:
        if "Sag" in key:
            sag_prompts[key] = prompt_response_map[key]
    return sag_prompts

    
prompt_response_map = read_grammar_and_create_map('referenceGrammar.xml')
sag_prompts = get_sag_prompts(prompt_response_map)

# Load Test Data

In [3]:
test_file_csv = "textProcessing_testKaldi.csv"
test_data = None
with open(test_file_csv, 'r') as reader:
    test_data = reader.readlines()  
    
def prompt_map_test_data():
    prompt_map = {}
    for item in test_data[1:]:
        split = item.replace("\n", "").split("\t")
        prompt = split[1]
        
        if prompt in prompt_map:
            prompt_map[prompt].append({'id': split[0], "transcript": split[3]})
        else:
            arr = []
            arr.append({'id':split[0], "transcript":split[3]})
            prompt_map[prompt] = arr
            
    return prompt_map
prompt_test_map = prompt_map_test_data()



In [42]:
def diff_of_two_maps(map_1, map_2):
    diff_map = {}
    for key in map_1:
        if key not in map_2:
            diff_map[key] = map_1[key]
    
    return diff_map

## Cluster Data auf Basis von Schritt 3.

In [408]:
def nlp_sentence_deutsch(sentence):
    parsed = nlp_de(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss

In [388]:
def id_to_prompt():
    id_prompt_map = {}
    for item in test_data[1:]:
        split = item.replace("\n", "").split("\t")
        prompt = split[1]
        id_ = split[0]
        id_prompt_map[id_] = prompt
            
    return id_prompt_map
id_prompt_map = id_to_prompt()

In [394]:
id_prompt_transcript_map = {}
for id_ in false_prompt_sc_to_nc:
    id_prompt_transcript_map[id_] = {"transcript": false_prompt_sc_to_nc[id_], "prompt": id_prompt_map[id_]}
    

In [409]:
nlp_sentence_deutsch("Ich habe keine Reservation")

([Ich, habe, keine, Reservation],
 ['ich', 'habe', 'keine', 'reservation'],
 ['PPER', 'VAFIN', 'PIAT', 'NN'],
 ['PPER', 'VAFIN', 'PIAT', 'NN'])

In [416]:
prompt_pos = {}
for id_ in id_prompt_transcript_map:
    key = id_prompt_transcript_map[id_]['prompt']
    if key in prompt_pos:
        prompt_pos[key] += 1
    else:
        prompt_pos[key] = 1
#id_prompt_transcript_map["4081"]

In [418]:
with open("cluster_test_pos_data.csv", "w") as writer:
    for prompt in prompt_pos:
        cleared_prompt = prompt.replace("Frag: ", "").replace("Sag: ", "").replace(".", "").replace("?", "")
        tags = nlp_sentence_deutsch(cleared_prompt)[2]
        writer.write(prompt + "\t" + " ".join(tags) + "\t" + str(prompt_pos[prompt]) + "\n")

# 1. Apply Reference Grammar

In [38]:
counter = 0

false_prompts_rg_base = {}
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue

        if sentence not in prompt_response_map[prompt_unit]:
            counter +=1
            false_prompts_rg_base[dict_prompt['id']] = sentence
                #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
        
len(false_prompts_rg_base)

461

# 2. Cut Whitespace

In [45]:
false_prompt_white_space_base = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            counter +=1
            false_prompt_white_space_base[dict_prompt['id']] = sentence
                #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

448

# 3. Special Case to Normal Case

In [46]:
special_case_map = {"don't": "do not",
 "haven't" : "have not",
 "i'd" : "i would",
 "i'm" : "i am",
 "isn't": "is not",
 "it's": "it is",
 "o'clock": "o'clock",
 "that's" :"that is",
 "there's": "there is"}

def sentence_to_normal_case(sentence):
    trans_sentence = ""
    words = sentence.split(" ")
    for word in words:
        if word in special_case_map:
            word = special_case_map[word]
        trans_sentence += word + " "
    
    return trans_sentence[:-1]

false_prompt_sc_to_nc = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            counter +=1
            false_prompt_sc_to_nc[dict_prompt['id']] = sentence
                #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

437

## 3.1 Diff between White Space Maps (2.) and Special Case to Normal Case (3.)

In [49]:
diff_of_two_maps(false_prompt_white_space_base, false_prompt_sc_to_nc)

{'3705': "i'm from america",
 '3801': "i'd like to pay with post card",
 '3909': "i'm fine",
 '3931': "i'm fine",
 '3940': "i'm fine",
 '3945': "i'm looking for trousers",
 '4273': "i'm fine",
 '4344': "i'm good",
 '4438': "i'm good",
 '4450': "it's too expensive",
 '4659': "i'm fine"}

## 3.2 Apply Diff Grammar from Training Data

In [421]:
import copy
def merge_response_map(grammar1, grammar2):
    grammar = copy.deepcopy(grammar1)
    for key in grammar1:
        if key in grammar2:            
            for item in grammar2[key]:
                arr = list(grammar[key])
                arr.append(item)
                grammar[key] = arr
            grammar[key] = set(grammar[key])
    return grammar

prompt_diff_map_2 = read_grammar_and_create_map("diff_rg_1.xml")
grammar = merge_response_map(prompt_response_map, prompt_diff_map_2)

In [423]:
counter = 0
new_grammar_with_diff = {}
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in grammar[prompt_unit]:
            counter +=1
            new_grammar_with_diff[dict_prompt['id']] = sentence
                #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

391

In [424]:
diff_of_two_maps(false_prompt_sc_to_nc, new_grammar_with_diff)

{'3651': 'this is my passport',
 '3660': 'i would like to buy four musical tickets',
 '3726': 'single room',
 '3727': 'single room',
 '3770': 'i have no reservation',
 '3773': 'i have not got a reservation',
 '3794': 'a twin room',
 '3810': 'i want to pay with credit card',
 '3854': 'the tower is near the tower bridge',
 '3861': 'where is the tower',
 '3883': 'i would like the bill',
 '3905': 'a ticket for starlight express',
 '3923': 'thirty eight',
 '3934': 'thirty nine',
 '3943': 'forty four',
 '3978': "i want to leave at eight o'clock tomorrow",
 '3981': 'this is okay',
 '4014': 'i have no reservation',
 '4089': 'forty one',
 '4090': 'forty one',
 '4091': 'i want some jeans',
 '4142': 'single room',
 '4144': 'single room',
 '4145': 'i want to pay by visa',
 '4152': 'in the room there is no shampoo',
 '4171': 'twin room',
 '4182': 'i could not find a hairdryer',
 '4187': 'no i have no reservation',
 '4207': 'i am on holidays',
 '4229': 'where is the tower',
 '4230': 'where is the to

# 4. Remove Yes/No 

In [239]:
yes_no_filtered_prompt = {}
counter =0

def remove_yes_no(sentence):
    if "yes" in sentence or \
    "no" in sentence or \
    "thanks" in sentence or \
    "thank you" in sentence or \
    "please" in sentence:
        sentence = sentence.replace("yes ", "")\
        .replace("no ", "")\
        .replace("thanks ", "")\
        .replace("thank you ", "")\
        .replace("please ", "")\
        .replace("also", "")
        #print(sentence)
    return sentence

for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            if remove_yes_no(sentence) not in prompt_response_map[prompt_unit]:
                counter +=1
                yes_no_filtered_prompt[dict_prompt['id']] = sentence
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

437

# 5. Unqiue Words

In [180]:
unique_prompts_res = {}
counter =0

def get_unique_sentence(sentence):
    trans_sentence = ""
    dict_words = {}
    words = sentence.split(" ")
    for word in words:
        if word in dict_words:
            continue
        dict_words[word] = 1
        trans_sentence += word + " "
    return trans_sentence[:-1]

for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    counter +=1
                    unique_prompts_res[dict_prompt['id']] = sentence
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

can i have not a room for one room
i would like to nothing hill
i would like to notting hill
where is not a a hairdryer
i would like tickets for the mastercard please
i do not note
i do not not
can i will have a single room please
this is my pass card
two like tickets please
i ticket please
i do not want the
this is not what i have a water
the not what i water
that is not what i bar
this is not what i bar
this is not what white i card
i have only big note
i want please
i want please
i could not find a hairdryer
i pay is there a reservation
i have reservation
i have not got a reservation
i have not a reservation
where i have not have i have no
i do not have a reservation any
i do not i do not have a reservation
i have reservation
i do not i do not have a reservation
i am good
i have reservation
i have not a reservation
i do not have a reservation is bern
i do not where is bern
i have not a reservation
the i do not have a reservation i pay by is my name is
i have not a reservation
i have

432

## 5.1 Unique Words (5.) compare Special Case to No Case (3.)

In [82]:
diff_of_two_maps(false_prompt_sc_to_nc, unique_prompts_res)

{'3676': 'i want a a double room',
 '3986': 'no i do not no i do not have a reservation',
 '4015': 'no i do not no i do not have a reservation',
 '4020': 'is there a a lift',
 '4533': 'can i pay with visa visa'}

# 6. Pos Map of RG

In [84]:
def nlp_sentence(sentence):
    parsed = parser(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss

In [130]:
def insert(tree, key, value):
    #print(key)
    if key:
        first, rest = key[0], key[1:]
        if first not in tree:
            tree[first] = {}
        insert(tree[first], rest, value)
    else:
        tree['key'] = True

In [131]:
tree = {}
tag_array = nlp_sentence("i want a a double room")[2]
insert(tree, tag_array, "true")

In [132]:
tree = {}
for item in prompt_response_map:
    
    for response in prompt_response_map[item]:
        tags = nlp_sentence(response)[2]
        insert(tree, tags, "true")

In [133]:
tree

{'CD': {'JJ': {'NN': {'UH': {'key': True}, 'key': True},
   'NNS': {'VBP': {'key': True}, 'key': True}},
  'NN': {'IN': {'DT': {'NN': {'IN': {'DT': {'NN': {'UH': {'key': True},
         'key': True}}},
      'NN': {'UH': {'key': True}, 'key': True},
      'UH': {'key': True},
      'key': True}},
    'JJ': {'NN': {'key': True}, 'UH': {'key': True}, 'key': True},
    'NN': {'NN': {'UH': {'key': True}, 'key': True},
     'UH': {'key': True},
     'key': True},
    'RB': {'NN': {'UH': {'key': True}, 'key': True}},
    'VB': {'key': True}},
   'UH': {'key': True},
   'key': True},
  'NNS': {'IN': {'DT': {'NN': {'IN': {'DT': {'NN': {'UH': {'key': True},
         'key': True}}},
      'NN': {'UH': {'key': True}, 'key': True},
      'UH': {'key': True},
      'key': True}},
    'JJ': {'UH': {'key': True}, 'key': True},
    'NN': {'NN': {'UH': {'key': True}, 'key': True},
     'UH': {'key': True},
     'key': True},
    'RB': {'NN': {'key': True}},
    'VB': {'key': True}},
   'UH': {'key': Tr

In [213]:
def existintree(tree, array):
    if len(array) == 0:
        return False
    if array[0] not in tree:
        return False
    if array[0] in tree:
        if 'key' in tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    return existintree(tree, array[1:])

#'CD': {'JJ': {'NN': {'UH'
#existintree(tree, ["CD", "asdf", "JJ", "NN"])

In [220]:
tag_array = nlp_sentence("i want a double room")[2]
#for tag in tag_array:
print(tag_array)

existintree(tree, tag_array)

['PRP', 'VBP', 'DT', 'JJ', 'NN']


True

In [221]:
exist_in_tree_map = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    if existintree(tree, nlp_sentence(unique_yes_no_sentence)[2]) == False:
                        counter +=1
                        exist_in_tree_map[dict_prompt['id']] = sentence
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter
#exist_in_tree_map

can i have not a room for one room
i would like to nothing hill
i would like to notting hill
where is not a a hairdryer
i would like tickets for the mastercard please
i do not note
i do not not
can i will have a single room please
this is my pass card
two like tickets please
i ticket please
i do not want the
this is not what i have a water
the not what i water
that is not what i bar
this is not what i bar
this is not what white i card
i have only big note
i want please
i want please
i could not find a hairdryer
i pay is there a reservation
i have reservation
i have not got a reservation
i have not a reservation
where i have not have i have no
i do not have a reservation any
i do not i do not have a reservation
i have reservation
i do not i do not have a reservation
i am good
i have reservation
i have not a reservation
i do not have a reservation is bern
i do not where is bern
i have not a reservation
the i do not have a reservation i pay by is my name is
i have not a reservation
i have

254

In [222]:
diff_of_two_maps(unique_prompts_res, exist_in_tree_map)

{'3574': 'have a fitness studio',
 '3595': 'pay a i would like pommes',
 '3602': 'i would like a hotel',
 '3608': 'i would like please',
 '3610': 'no i have you the gym',
 '3621': 'i am a',
 '3634': 'i ticket please',
 '3637': 'have a room',
 '3642': 'this is there the hotel bar',
 '3647': 'i have a brothers',
 '3648': 'i have a brown boots',
 '3651': 'this is my passport',
 '3671': 'no i pay is there a reservation',
 '3679': 'have you have a pool',
 '3712': 'where is my passport',
 '3713': 'that is like post card',
 '3716': 'i want pay',
 '3726': 'single room',
 '3727': 'single room',
 '3729': 'i do not note',
 '3768': 'is there a room',
 '3770': 'i have no reservation',
 '3773': 'i have not got a reservation',
 '3774': 'i have not a reservation',
 '3779': 'where no i have not have i have no',
 '3787': 'is there a bar',
 '3789': 'where is there a bar',
 '3794': 'a twin room',
 '3804': 'i would like visa',
 '3807': 'no i do not have a reservation any',
 '3817': 'the is my passport',
 '

## 6.1 Singular Plural Number Rule

In [ ]:
import copy

In [359]:
def existintree2(tree, array, rest_tree):
    if len(array) == 0:
        return False
    if array[0] not in rest_tree:
        return False
    if array[0] in tree:
        if 'key' in tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    
    return existintree2(tree, array[1:], rest_tree[array[0]])

#print(existintree2(tree, nlp_sentence('can i pay by card')[2], copy.deepcopy(tree)))

def sing_plu(nlp_sent):
    words = nlp_sent[0]
    tags = nlp_sent[2]
    
    if 'CD' in tags:
        index_cd = tags.index('CD')
        if str(words[index_cd]) == 'one':
            if 'NNS' not in tags:
                return True
            else:
                return False
        else:
            if 'NN' not in tags:
                return True
            else:
                return False
    
    return True

sing_plu_map = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    nlp_sent = nlp_sentence(unique_yes_no_sentence)
                    tags = nlp_sent[2]
                    # POS zwar bekannt jedoch nicht sicher
                    if existintree2(tree, tags, copy.deepcopy(tree)) == True:
                        # Überprüfe Singular
                        if sing_plu(nlp_sent) == False:
                            counter +=1
                            sing_plu_map[dict_prompt['id']] = {"sentence": sentence, "reason": "singplural"}
                            #sing_plu_map['reason'] = "sing_plural"
                    # POS unbekannt = "rejected"
                    else:
                        counter +=1
                        sing_plu_map[dict_prompt['id']] = {"sentence": sentence, "reason": "tag"}
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter
#exist_in_tree_map

264

In [363]:
def existintree4(tree, array, rest_tree):
    if len(array) == 0:
        return False
    if array[0] not in rest_tree:
        return False
    if array[0] in rest_tree:
        if 'key' in rest_tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    
    return existintree4(tree, array[1:], rest_tree[array[0]])

#print(existintree2(tree, nlp_sentence('can i pay by card')[2], copy.deepcopy(tree)))

def sing_plu(nlp_sent):
    words = nlp_sent[0]
    tags = nlp_sent[2]
    
    if 'CD' in tags:
        index_cd = tags.index('CD')
        if str(words[index_cd]) == 'one':
            if 'NNS' not in tags:
                return True
            else:
                return False
        else:
            if 'NN' not in tags:
                return True
            else:
                return False
    
    return True

sing_plu_map_2 = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    nlp_sent = nlp_sentence(unique_yes_no_sentence)
                    tags = nlp_sent[2]
                    # POS zwar bekannt jedoch nicht sicher
                    if existintree4(tree, tags, copy.deepcopy(tree)) == True:
                        # Überprüfe Singular
                        if sing_plu(nlp_sent) == False:
                            counter +=1
                            sing_plu_map_2[dict_prompt['id']] = {"sentence": sentence, "reason": "singplural"}
                            #sing_plu_map['reason'] = "sing_plural"
                    # POS unbekannt = "rejected"
                    else:
                        counter +=1
                        sing_plu_map_2[dict_prompt['id']] = {"sentence": sentence, "reason": "tag"}
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter
#exist_in_tree_map

258

In [364]:
diff_of_two_maps(sing_plu_map, sing_plu_map_2)

{'3763': {'reason': 'singplural',
  'sentence': 'i want a room for seven nights'},
 '3803': {'reason': 'tag', 'sentence': 'i am franc'},
 '3830': {'reason': 'tag', 'sentence': 'i want a ticket for the musical'},
 '3919': {'reason': 'tag', 'sentence': 'that is too expensive'},
 '3960': {'reason': 'tag', 'sentence': 'my name is sarah'},
 '3981': {'reason': 'tag', 'sentence': 'this is okay'},
 '4002': {'reason': 'tag', 'sentence': 'i need a ticket for this musical'},
 '4158': {'reason': 'singplural', 'sentence': 'four tickets for london'},
 '4186': {'reason': 'tag', 'sentence': 'no i am good'},
 '4219': {'reason': 'singplural',
  'sentence': 'can i have a room for four nights'},
 '4405': {'reason': 'tag', 'sentence': 'the capital of switzerland is rome'},
 '4442': {'reason': 'singplural', 'sentence': 'four tickets for the room'},
 '4453': {'reason': 'tag', 'sentence': 'i would like boots row'},
 '4512': {'reason': 'tag', 'sentence': 'the capital of austria is wien'},
 '4608': {'reason': '

In [365]:
sing_plu_map_2

{'3570': {'reason': 'tag',
  'sentence': 'can i have a hotel room for one nights'},
 '3574': {'reason': 'tag', 'sentence': 'have a fitness studio'},
 '3584': {'reason': 'tag',
  'sentence': 'i would like a i would like to london eye for i hotel bar'},
 '3595': {'reason': 'tag', 'sentence': 'pay a i would like pommes'},
 '3597': {'reason': 'tag', 'sentence': 'can i am a pay by mastercard'},
 '3608': {'reason': 'tag', 'sentence': 'i would like please'},
 '3610': {'reason': 'tag', 'sentence': 'no i have you the gym'},
 '3620': {'reason': 'tag',
  'sentence': 'i want to leave at my passport tomorrow'},
 '3621': {'reason': 'tag', 'sentence': 'i am a'},
 '3623': {'reason': 'tag', 'sentence': 'two like tickets please'},
 '3630': {'reason': 'tag', 'sentence': 'i would like to sit in'},
 '3631': {'reason': 'tag', 'sentence': 'i would sit in the'},
 '3634': {'reason': 'tag', 'sentence': 'i ticket please'},
 '3637': {'reason': 'tag', 'sentence': 'have a room'},
 '3639': {'reason': 'tag', 'sentenc

## Dif 6.2 zu 6.1 (6.2 hat mehr deswegen besser wegen False Positives)

In [313]:
def diff_of_two_maps(map_1, map_2):
    diff_map = {}
    for key in map_1:
        if key not in map_2:
            diff_map[key] = map_1[key]
    
    return diff_map

In [316]:
diff_of_two_maps(sing_plu_map, exist_in_tree_map)

{'3574': {'reason': 'tag', 'sentence': 'have a fitness studio'},
 '3595': {'reason': 'tag', 'sentence': 'pay a i would like pommes'},
 '3608': {'reason': 'tag', 'sentence': 'i would like please'},
 '3610': {'reason': 'tag', 'sentence': 'no i have you the gym'},
 '3634': {'reason': 'tag', 'sentence': 'i ticket please'},
 '3637': {'reason': 'tag', 'sentence': 'have a room'},
 '3642': {'reason': 'tag', 'sentence': 'this is there the hotel bar'},
 '3648': {'reason': 'tag', 'sentence': 'i have a brown boots'},
 '3651': {'reason': 'tag', 'sentence': 'this is my passport'},
 '3671': {'reason': 'tag', 'sentence': 'no i pay is there a reservation'},
 '3712': {'reason': 'tag', 'sentence': 'where is my passport'},
 '3713': {'reason': 'tag', 'sentence': 'that is like post card'},
 '3716': {'reason': 'tag', 'sentence': 'i want pay'},
 '3773': {'reason': 'tag', 'sentence': 'i have not got a reservation'},
 '3774': {'reason': 'tag', 'sentence': 'i have not a reservation'},
 '3779': {'reason': 'tag', 

In [317]:
sing_plu_map

{'3570': {'reason': 'tag',
  'sentence': 'can i have a hotel room for one nights'},
 '3574': {'reason': 'tag', 'sentence': 'have a fitness studio'},
 '3584': {'reason': 'tag',
  'sentence': 'i would like a i would like to london eye for i hotel bar'},
 '3595': {'reason': 'tag', 'sentence': 'pay a i would like pommes'},
 '3597': {'reason': 'tag', 'sentence': 'can i am a pay by mastercard'},
 '3608': {'reason': 'tag', 'sentence': 'i would like please'},
 '3610': {'reason': 'tag', 'sentence': 'no i have you the gym'},
 '3620': {'reason': 'tag',
  'sentence': 'i want to leave at my passport tomorrow'},
 '3623': {'reason': 'tag', 'sentence': 'two like tickets please'},
 '3630': {'reason': 'tag', 'sentence': 'i would like to sit in'},
 '3634': {'reason': 'tag', 'sentence': 'i ticket please'},
 '3637': {'reason': 'tag', 'sentence': 'have a room'},
 '3639': {'reason': 'tag', 'sentence': 'i want to pay with my post card'},
 '3640': {'reason': 'tag',
  'sentence': 'i would like to pay with by po

In [194]:
print(nlp_sentence("can i have a hotel room for one nights")[2])
len(tree['MD']['PRP']['VB']['DT']['NN']['NN']['UH'])

['MD', 'PRP', 'VB', 'DT', 'NN', 'NN', 'IN', 'CD', 'NNS']


1

In [248]:
nlp_sentence("can i have a hotel room for one nights")

([can, i, have, a, hotel, room, for, one, nights],
 ['can', 'i', 'have', 'a', 'hotel', 'room', 'for', 'one', 'night'],
 ['MD', 'PRP', 'VB', 'DT', 'NN', 'NN', 'IN', 'CD', 'NNS'],
 ['MD', 'PRP', 'VB', 'DT', 'NN', 'NN', 'IN', 'CD', 'NNS'])

# 6.2 Chunk Prepo

In [346]:
def existintree3(tree, array, rest_tree, index=0):
    if len(array) == 0:
        return index
    if array[0] not in rest_tree:
        return index
    if array[0] in tree:
        if 'key' in tree[array[0]] and len(array) == 1:
            return index
    #print(tree[array[0]])
    index +=1
    return existintree3(tree, array[1:], rest_tree[array[0]], index)

def sing_plu(nlp_sent):
    words = nlp_sent[0]
    tags = nlp_sent[2]
    
    if 'CD' in tags:
        index_cd = tags.index('CD')
        if str(words[index_cd]) == 'one':
            if 'NNS' not in tags:
                return True
            else:
                return False
        else:
            if len(words) > index_cd + 1 and str(tags[index_cd+1]) == 'NNS':
                return True
            
            elif 'NN' not in tags:
                return True
            else:
                return False
    
    return True

prepo_map = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    nlp_sent = nlp_sentence(unique_yes_no_sentence)
                    tags = nlp_sent[2]
                    # POS zwar bekannt jedoch nicht sicher
                    if existintree2(tree, tags, copy.deepcopy(tree)) == True:
                        # Überprüfe Singular
                        if sing_plu(nlp_sent) == False:
                            counter +=1
                            prepo_map[dict_prompt['id']] = {"sentence": sentence, "reason": "singplural"}
                            #sing_plu_map['reason'] = "sing_plural"
                    # POS unbekannt = "rejected"
                    else:
                        index = existintree3(tree, tags, copy.deepcopy(tree))
                        if nlp_sent[2] in chunks and sing_plu(nlp_sent):
                            print(sentence)
                            continue
                        counter +=1
                        prepo_map[dict_prompt['id']] = {"sentence": sentence, "reason": "tag"}
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

in the room is
like a ticket to piccadilly circus


257

In [348]:
diff_of_two_maps(sing_plu_map, prepo_map)

{'3655': {'reason': 'tag', 'sentence': 'like a ticket to piccadilly circus'},
 '3763': {'reason': 'singplural',
  'sentence': 'i want a room for seven nights'},
 '4047': {'reason': 'tag', 'sentence': 'in the room is'},
 '4158': {'reason': 'singplural', 'sentence': 'four tickets for london'},
 '4219': {'reason': 'singplural',
  'sentence': 'can i have a room for four nights'},
 '4442': {'reason': 'singplural', 'sentence': 'four tickets for the room'},
 '4611': {'reason': 'singplural',
  'sentence': 'can i have a room for four nights'}}

In [349]:
sing_plu_map

{'3570': {'reason': 'tag',
  'sentence': 'can i have a hotel room for one nights'},
 '3574': {'reason': 'tag', 'sentence': 'have a fitness studio'},
 '3584': {'reason': 'tag',
  'sentence': 'i would like a i would like to london eye for i hotel bar'},
 '3595': {'reason': 'tag', 'sentence': 'pay a i would like pommes'},
 '3597': {'reason': 'tag', 'sentence': 'can i am a pay by mastercard'},
 '3608': {'reason': 'tag', 'sentence': 'i would like please'},
 '3610': {'reason': 'tag', 'sentence': 'no i have you the gym'},
 '3620': {'reason': 'tag',
  'sentence': 'i want to leave at my passport tomorrow'},
 '3623': {'reason': 'tag', 'sentence': 'two like tickets please'},
 '3630': {'reason': 'tag', 'sentence': 'i would like to sit in'},
 '3634': {'reason': 'tag', 'sentence': 'i ticket please'},
 '3637': {'reason': 'tag', 'sentence': 'have a room'},
 '3639': {'reason': 'tag', 'sentence': 'i want to pay with my post card'},
 '3640': {'reason': 'tag',
  'sentence': 'i would like to pay with by po

In [376]:
#if 'CD' = "one" then no 'NNS':
#    else: then no 'NN'

def existintree4(tree, array, rest_tree):
    if len(array) == 0:
        return False
    if array[0] not in rest_tree:
        return False
    if array[0] in rest_tree:
        if 'key' in rest_tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    
    return existintree4(tree, array[1:], rest_tree[array[0]])

sent = "can i have a hotel room for one nights"
sent = "i want a room for seven nights"
sent = "i want a ticket for the musical"
sent = "the capital of switzerland is rome"
sent = "can i have a hotel room for one nights"
sent = "i would like to buy four musical tickets"
#sent = "can i have a hotel room"
nlp_s = nlp_sentence(sent)
print(nlp_s)
print(existintree4(tree, nlp_s[2], copy.deepcopy(tree)))
#print(nlp_s[0][6:])
#print(sing_plu(nlp_sentence(sent)))
#diff_of_two_maps(unique_prompts_res, exist_in_tree_map)

([i, would, like, to, buy, four, musical, tickets], ['i', 'would', 'like', 'to', 'buy', 'four', 'musical', 'ticket'], ['PRP', 'MD', 'VB', 'TO', 'VB', 'CD', 'JJ', 'NNS'], ['PRP', 'MD', 'VB', 'TO', 'VB', 'CD', 'JJ', 'NNS'])
False


In [380]:
tree['PRP'][ 'MD'][ 'VB'][ 'TO'][ 'VB']

{'DT': {'JJ': {'NN': {'UH': {'key': True}, 'key': True}},
  'NN': {'IN': {'DT': {'NN': {'IN': {'DT': {'NN': {'UH': {'key': True},
         'key': True}}},
      'UH': {'key': True},
      'key': True}},
    'NN': {'NN': {'UH': {'key': True}, 'key': True},
     'UH': {'key': True},
     'key': True},
    'RB': {'NN': {'UH': {'key': True}, 'key': True}},
    'VBG': {'NN': {'UH': {'key': True}, 'key': True}}},
   'NN': {'NN': {'UH': {'key': True}, 'key': True},
    'UH': {'key': True},
    'key': True},
   'PRP': {'VBD': {'UH': {'key': True}, 'key': True}},
   'UH': {'key': True},
   'WDT': {'PRP': {'VBD': {'UH': {'key': True}, 'key': True}}},
   'key': True},
  'NNP': {'NN': {'UH': {'key': True}, 'key': True}},
  'NNS': {'UH': {'key': True}, 'key': True},
  'RB': {'NN': {'UH': {'key': True}, 'key': True}},
  'VBN': {'NN': {'UH': {'key': True}, 'key': True}}},
 'IN': {'DT': {'JJ': {'NN': {'key': True}, 'key': True},
   'NN': {'IN': {'CD': {'NN': {'key': True}, 'NNS': {'key': True}}},
    

In [373]:
sent = "can i have a hotel room for one nights"
#sent = "can i have a hotel room"
nlp_s = nlp_sentence(sent)
index = existintree3(tree, tags, copy.deepcopy(tree))
if nlp_s[2] in chunks and sing_plu(nlp_sent):
    print(sentence)

In [325]:
pre_chunks = {'IN',
 'IN CD JJ NN',
 'IN CD NN',
 'IN CD NN NN',
 'IN CD NN NN SP NN',
 'IN CD NNS',
 'IN CD NNS . SP NN',
 'IN CD NNS IN VB',
 'IN CD NNS UH',
 'IN CD NNS VBZ PRP DT SP NN VBP CD DT SP PRP VBP DT NN IN CD NNS',
 'IN CD SP',
 'IN CD SP NFP NFP NFP SP NFP NFP NFP',
 'IN CD SP NFP NFP NFP SP NNS',
 'IN CD TO VB NN',
 'IN DT',
 'IN DT JJ',
 'IN DT JJ NN',
 'IN DT JJ NN NN SP NFP NFP NFP',
 'IN DT JJ NN VBZ EX VBZ EX',
 'IN DT NN',
 'IN DT NN DT',
 'IN DT NN EX VBZ DT NN',
 'IN DT NN IN CD NNS',
 'IN DT NN IN DT NN',
 'IN DT NN IN NN',
 'IN DT NN IN RB NN',
 'IN DT NN NN',
 'IN DT NN NN SP',
 'IN DT NN SP',
 'IN DT NN UH',
 'IN DT NN VBZ',
 'IN DT NN VBZ RB DT SP NFP NFP NFP SP NN',
 'IN DT NN VBZ VBN',
 'IN DT NNS IN NN',
 'IN DT PRP SP VBP DT SP JJ NN IN WRB VBP EX DT SP IN DT SP NN IN NN NN NN',
 'IN DT RB DT SP TO VB NN NN',
 'IN DT SP NFP NFP NFP SP NN',
 'IN DT SP NN',
 'IN DT VBZ DT NN',
 'IN IN NN NN',
 'IN IN NN NNS',
 'IN IN SP NFP NFP NFP SP NN IN NN',
 'IN JJ',
 'IN JJ NN',
 'IN JJ NN NNS',
 'IN JJ NN VBP SP',
 'IN JJ NNS',
 'IN NN',
 'IN NN IN NN',
 'IN NN IN NN NN',
 'IN NN IN NN SP',
 'IN NN NN',
 'IN NN NN PRP SP MD PRP VB DT NN IN NN NN',
 'IN NN NN SP',
 'IN NN NNS',
 'IN NN SP',
 'IN NN SP NFP NFP NFP',
 'IN NN TO VB SP',
 'IN NN VBN NN',
 'IN NN VBZ IN IN SP NFP NFP NFP SP NN IN NN',
 'IN NN VBZ JJ',
 'IN NN VBZ NN',
 'IN NN VBZ UH',
 'IN NN VBZ VBN',
 'IN NNP',
 'IN NNP VBZ NN',
 'IN NNS',
 'IN NNS VBP',
 'IN PRP',
 'IN PRP SP NN NN',
 'IN PRP SP VBP IN NNP DT NN',
 'IN PRP$ NN',
 'IN PRP$ NN NN',
 'IN PRP$ NN NN NN',
 'IN PRP$ NN VBZ NN',
 'IN PRP$ SP NFP NFP NFP SP NN',
 'IN RB JJ NN',
 'IN RB NN',
 'IN SP NFP NFP NFP SP CD NNS',
 'IN SP NFP NFP NFP SP DT NN',
 'IN SP NFP NFP NFP SP IN NN NN',
 'IN SP NFP NFP NFP SP JJ NN',
 'IN SP NFP NFP NFP SP NN',
 'IN SP NFP NFP NFP SP NN IN NN',
 'IN SP NFP NFP NFP SP NN NN',
 'IN SP NFP NFP NFP SP NN PRP',
 'IN SP NFP NFP NFP SP VB NN',
 'IN TO VB',
 'IN TO VB DT SP NN',
 'IN UH NN IN NN NN NN',
 'IN VB',
 'IN VBG IN NN NN',
 'IN VBG NN',
 'IN VBG SP NFP NFP NFP',
 'IN VBP DT SP NN',
 'IN VBP DT SP NN SP',
 'IN VBP SP NFP NFP NFP SP NNS IN NN NN',
 'IN VBZ DT NN',
 'IN VBZ EX DT SP IN DT SP NN',
 'IN VBZ JJ'}

['IN']

In [260]:
tree['MD']['PRP']['VB']['IN']['NN']['key']

True

In [282]:
if 'PRP' in tree['MD']:
    print("he")

he


In [324]:
pre_chunks = {'IN',
 'IN CD JJ NN',
 'IN CD NN',
 'IN CD NN NN',
 'IN CD NN NN SP NN',
 'IN CD NNS',
 'IN CD NNS . SP NN',
 'IN CD NNS IN VB',
 'IN CD NNS UH',
 'IN CD NNS VBZ PRP DT SP NN VBP CD DT SP PRP VBP DT NN IN CD NNS',
 'IN CD SP',
 'IN CD SP NFP NFP NFP SP NFP NFP NFP',
 'IN CD SP NFP NFP NFP SP NNS',
 'IN CD TO VB NN',
 'IN DT',
 'IN DT JJ',
 'IN DT JJ NN',
 'IN DT JJ NN NN SP NFP NFP NFP',
 'IN DT JJ NN VBZ EX VBZ EX',
 'IN DT NN',
 'IN DT NN DT',
 'IN DT NN EX VBZ DT NN',
 'IN DT NN IN CD NNS',
 'IN DT NN IN DT NN',
 'IN DT NN IN NN',
 'IN DT NN IN RB NN',
 'IN DT NN NN',
 'IN DT NN NN SP',
 'IN DT NN SP',
 'IN DT NN UH',
 'IN DT NN VBZ',
 'IN DT NN VBZ RB DT SP NFP NFP NFP SP NN',
 'IN DT NN VBZ VBN',
 'IN DT NNS IN NN',
 'IN DT PRP SP VBP DT SP JJ NN IN WRB VBP EX DT SP IN DT SP NN IN NN NN NN',
 'IN DT RB DT SP TO VB NN NN',
 'IN DT SP NFP NFP NFP SP NN',
 'IN DT SP NN',
 'IN DT VBZ DT NN',
 'IN IN NN NN',
 'IN IN NN NNS',
 'IN IN SP NFP NFP NFP SP NN IN NN',
 'IN JJ',
 'IN JJ NN',
 'IN JJ NN NNS',
 'IN JJ NN VBP SP',
 'IN JJ NNS',
 'IN NN',
 'IN NN IN NN',
 'IN NN IN NN NN',
 'IN NN IN NN SP',
 'IN NN NN',
 'IN NN NN PRP SP MD PRP VB DT NN IN NN NN',
 'IN NN NN SP',
 'IN NN NNS',
 'IN NN SP',
 'IN NN SP NFP NFP NFP',
 'IN NN TO VB SP',
 'IN NN VBN NN',
 'IN NN VBZ IN IN SP NFP NFP NFP SP NN IN NN',
 'IN NN VBZ JJ',
 'IN NN VBZ NN',
 'IN NN VBZ UH',
 'IN NN VBZ VBN',
 'IN NNP',
 'IN NNP VBZ NN',
 'IN NNS',
 'IN NNS VBP',
 'IN PRP',
 'IN PRP SP NN NN',
 'IN PRP SP VBP IN NNP DT NN',
 'IN PRP$ NN',
 'IN PRP$ NN NN',
 'IN PRP$ NN NN NN',
 'IN PRP$ NN VBZ NN',
 'IN PRP$ SP NFP NFP NFP SP NN',
 'IN RB JJ NN',
 'IN RB NN',
 'IN SP NFP NFP NFP SP CD NNS',
 'IN SP NFP NFP NFP SP DT NN',
 'IN SP NFP NFP NFP SP IN NN NN',
 'IN SP NFP NFP NFP SP JJ NN',
 'IN SP NFP NFP NFP SP NN',
 'IN SP NFP NFP NFP SP NN IN NN',
 'IN SP NFP NFP NFP SP NN NN',
 'IN SP NFP NFP NFP SP NN PRP',
 'IN SP NFP NFP NFP SP VB NN',
 'IN TO VB',
 'IN TO VB DT SP NN',
 'IN UH NN IN NN NN NN',
 'IN VB',
 'IN VBG IN NN NN',
 'IN VBG NN',
 'IN VBG SP NFP NFP NFP',
 'IN VBP DT SP NN',
 'IN VBP DT SP NN SP',
 'IN VBP SP NFP NFP NFP SP NNS IN NN NN',
 'IN VBZ DT NN',
 'IN VBZ EX DT SP IN DT SP NN',
 'IN VBZ JJ'}

chunks = []
for item in pre_chunks:
    chunk = item.split(" ")
    chunks.append(chunk)

# Stand heute Abend

In [383]:
def existintree3(tree, array, rest_tree, index=0):
    if len(array) == 0:
        return index
    if array[0] not in rest_tree:
        return index
    if array[0] in tree:
        if 'key' in tree[array[0]] and len(array) == 1:
            return index
    #print(tree[array[0]])
    index +=1
    return existintree3(tree, array[1:], rest_tree[array[0]], index)

def sing_plu(nlp_sent):
    words = nlp_sent[0]
    tags = nlp_sent[2]
    if 'CD' in tags:
        index_cd = tags.index('CD')
        if str(words[index_cd]) == 'one':
            if 'NNS' not in tags:
                return True
            else:
                return False
        else:
            if len(words) > index_cd + 1 and str(tags[index_cd+1]) == 'NNS':
                return True
            
            elif 'NN' not in tags:
                return True
            else:
                return False
    
    return True

def existintree4(tree, array, rest_tree):
    if len(array) == 0:
        return False
    if array[0] not in rest_tree:
        return False
    if array[0] in rest_tree:
        if 'key' in rest_tree[array[0]] and len(array) == 1:
            return True
    #print(tree[array[0]])
    
    return existintree4(tree, array[1:], rest_tree[array[0]])


prepo_map = {}
prepo_map_correct = {}
counter =0
for prompt_unit in prompt_test_map:
    for dict_prompt in prompt_test_map[prompt_unit]:
        
        sentence = dict_prompt['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            yes_no_sentence = remove_yes_no(sentence)
            if yes_no_sentence not in prompt_response_map[prompt_unit]:
                
                unique_sentence = get_unique_sentence(sentence)
                unique_yes_no_sentence = get_unique_sentence(yes_no_sentence)
                if unique_sentence not in prompt_response_map[prompt_unit] or unique_yes_no_sentence not in prompt_response_map[prompt_unit]:
                    nlp_sent = nlp_sentence(unique_yes_no_sentence)
                    tags = nlp_sent[2]
                    # POS zwar bekannt jedoch nicht sicher
                    if existintree4(tree, tags, copy.deepcopy(tree)) == True:
                        # Überprüfe Singular
                        if sing_plu(nlp_sent) == False:
                            counter +=1
                            prepo_map[dict_prompt['id']] = {"sentence": sentence, "reason": "singplural"}
                            #sing_plu_map['reason'] = "sing_plural"
                        else:
                            prepo_map_correct[dict_prompt['id']] = sentence

                    # POS unbekannt = "rejected"
                    else:
                        """index = existintree3(tree, tags, copy.deepcopy(tree))"""
                        """if nlp_sent[2] in chunks and sing_plu(nlp_sent):
                            print(sentence)
                            continue"""
                        counter +=1
                        prepo_map[dict_prompt['id']] = {"sentence": sentence, "reason": "tag"}
                else:
                    prepo_map_correct[dict_prompt['id']] = sentence
            else:
                    prepo_map_correct[dict_prompt['id']] = sentence
                #if sentence == "Sag: Ich möchte mit Kreditkarte bezahlen":
                    #writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
counter

258

In [385]:
prepo_map_correct

{'3583': 'i would like tickets for the mastercard please',
 '3587': 'i would like a ticket to the gym',
 '3593': 'can i pay by card',
 '3594': 'can i pay by card',
 '3599': 'can i buy by post',
 '3602': 'i would like a hotel',
 '3604': 'i want to pay',
 '3618': 'the ticket to london',
 '3632': 'i would like to sit in the sixth row',
 '3647': 'i have a brothers',
 '3654': 'a ticket from piccadilly circus',
 '3676': 'i want a a double room',
 '3710': 'i go into holiday',
 '3726': 'single room',
 '3727': 'single room',
 '3729': 'i do not note',
 '3738': 'i want to pay with credit card',
 '3740': 'have you got a fitness studio',
 '3763': 'i want a room for seven nights',
 '3768': 'is there a room',
 '3770': 'i have no reservation',
 '3787': 'is there a bar',
 '3794': 'a twin room',
 '3795': 'i would like to pay by master card',
 '3796': 'i would like to pay with master with master card',
 '3803': 'i am franc',
 '3804': 'i would like visa',
 '3810': 'i want to pay with credit card',
 '3819'

In [427]:
def is_slice_in_list(s,l):
    len_s = len(s) #so we don't recompute length of s on every iteration
    return any(s == l[i:len_s+i] for i in range(len(l) - len_s+1))

def is_slice_in_list2(s,l):
    len_s = len(s) #so we don't recompute length of s on every iteration
    for i in range(len(l) - len_s+1):
        if s==l[i:len_s+i]:
            return i

l = ["NN", "ST", "NN","NN"]
s = ["NN", "NN"]
is_slice_in_list2(s, l)

2